# Task 3

In [0]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from sparkmeasure import StageMetrics
from pyspark.sql import functions as F
from operator import add
import operator
from pyspark.sql.functions import broadcast
from pyspark.sql.types import StringType
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
from operator import add

spark = SparkSession.builder.appName("task1").getOrCreate()

stagemetrics = StageMetrics(spark)
dbfs_fileStore_prefix = "/FileStore/tables"
prefix = "ontimeperformance"

In [0]:
def clean_column_names(df):
  tempList = [] #Edit01
  for col in df.columns:
      new_name = col.strip()
      new_name = "".join(new_name.split())
      new_name = new_name.replace('.','') 
      tempList.append(new_name) 

  return df.toDF(*tempList) 

In [0]:
def print_models(airlines):
    if len(airlines) == 0: return
    i = 0
    curr = airlines[0][0]
    has_print = False
    aircraft_type = ''
    while i < len(airlines):
        if curr == airlines[i][0]:
            aircraft_type += airlines[i][1][0] + " " + airlines[i][1][1] + ", "
            i += 1
        else:
            has_print = True
            print(curr + " \t " +  "[" + aircraft_type[:-2] + "]")
            curr = airlines[i][0]
            aircraft_type = ""
    if not has_print: print(curr + " \t " + "[" + aircraft_type[:-2] + "]")
  
def task_3_rdd(spark_session, flights_path, airlines_path, aircrafts_path, country):
    flights_rdd = spark.sparkContext.textFile(flights_path).map(lambda x: x.split(",")) 
    airlines_rdd = spark.sparkContext.textFile(airlines_path).map(lambda x: x.split(",")) 
    aircrafts_rdd = spark.sparkContext.textFile(aircrafts_path).map(lambda x: x.split(",")) 
    
    # remove null
    flights_rdd = flights_rdd.filter(lambda x: x[0] != "")
    flights_rdd = flights_rdd.filter(lambda x: x[1] != "")
    flights_rdd = flights_rdd.filter(lambda x: x[2] != "")
    # remove null
    airlines_rdd = airlines_rdd.filter(lambda x: x[0] != "")
    airlines_rdd = airlines_rdd.filter(lambda x: x[1] != "")
    airlines_rdd = airlines_rdd.filter(lambda x: x[2] != "")
    
    # mapping (carrier_code, (name,country))
    airlines_rdd = airlines_rdd.filter(lambda x: x[2] == country)
    airlines_rdd = airlines_rdd.map(lambda x: (x[0], (x[1], x[2])))
    
    # removing header and rows with null
    header = aircrafts_rdd.first()
    aircrafts_rdd = aircrafts_rdd.filter(lambda x: len(x) == len(header))
    aircrafts_rdd = aircrafts_rdd.filter(lambda x: x != header)
    
    # tailnum, manufacturer, model
    aircrafts_rdd = aircrafts_rdd.map(lambda x: (x[0], x[2], x[4]))
    
    # re mapping (carrier_code , tailnum)
    flights_rdd = flights_rdd.map(lambda x: (x[0], x[5]))
    
    # carrier_code, (name, country) , tailnum
    airlines_count = airlines_rdd.join(flights_rdd)
    
    # map (tailnum, (name, carrier_code))
    airlines_count = airlines_count.map(lambda x: (x[1][1], (x[1][0][0], x[0])))
    
    # re-shape ((tailnum, name, carrier_code), 1)
    airlines_count = airlines_count.map(lambda x: ((x[0], x[1][1], x[1][0]), 1))
    airlines_count = airlines_count.reduceByKey(lambda x, y: x + y)
    
    # (tailnum, (name, count))
    airlines_count = airlines_count.map(lambda x: (x[0][0], (x[0][2], x[1])))
    
    # tailnum, (manufacture, model)
    aircrafts_rdd = aircrafts_rdd.map(lambda x: (x[0], (x[1], x[2])))  

    airlines_count = airlines_count.join(aircrafts_rdd)
    # (name, manufacture, model), count
    airlines_count = airlines_count.map(
        lambda x: ((x[1][0][0], x[1][1][0], x[1][1][1]), x[1][0][1])
    )
    # (name, manfufacturer, model), sum(count)
    airlines_count = airlines_count.reduceByKey(lambda x,y:x+y)    
    airlines_count = airlines_count.map(
        lambda x: (x[0][0], (x[0][1], x[0][2], x[0][2], x[1]))
    )
    
    # sorting by count then by airline name
    airlines_count = airlines_count.sortBy(lambda x: x[1][3], False).sortByKey()
    airlines = airlines_count.collect()
    
    # take top 5 by airline
    i = 0
    curr = airlines[i][0]
    aircraft = 0
    aircraft_rank = []
    
    while i < len(airlines):
        if curr == airlines[i][0]:
            # Top 5
            if aircraft < 5:
                aircraft_rank.append(airlines[i])
                i += 1
                aircraft += 1
            else: i += 1
        else:
            aircraft = 0
            curr = airlines[i][0]
    
    # output top 5 aircrafts per airline
    print_models(aircraft_rank)

In [0]:
stagemetrics.begin()

task_3_rdd(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_small.csv", 
                                f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv", 
                                f"{dbfs_fileStore_prefix}/{prefix}_aircrafts.csv", "United States")
stagemetrics.end()

AirTran [BOEING 717-200, BOEING 737-7BD, BOEING 737-76N]
Alaska Airlines Inc. [BOEING 737-4Q8, BOEING 737-490, BOEING 737-890, BOEING 737-790, BOEING 737-990]
American Airlines Inc. [MCDONNELL DOUGLAS DC-9-82(MD-82), MCDONNELL DOUGLAS DC-9-83(MD-83), BOEING 757-223, AIRBUS INDUSTRIE A319-112, AIRBUS INDUSTRIE A319-132]
American Eagle Airlines Inc. [EMBRAER EMB-145LR, EMBRAER EMB-135KL, EMBRAER EMB-135LR, BOMBARDIER INC CL-600-2C10, SAAB-SCANIA SAAB 340B]
Atlantic Southeast Airlines [BOMBARDIER INC CL-600-2B19, EMBRAER EMB-145LR, BOMBARDIER INC CL-600-2C10, EMBRAER EMB-145XR, CANADAIR CL-600-2B19]
Comair [BOMBARDIER INC CL-600-2B19, CANADAIR CL-600-2B19, BOMBARDIER INC CL-600-2C10, BOMBARDIER INC CL600-2D24, PIPER PA-28-180]
Continental Air Lines Inc. [BOEING 737-524, BOEING 737-824, BOEING 737-3TO, BOEING 737-724, BOEING 757-224]
Delta Air Lines Inc. [MCDONNELL DOUGLAS AIRCRAFT CO MD-88, BOEING 757-232, BOEING 737-832, BOEING 717-200, BOEING 767-332]
Frontier Airlines Inc. [AIRBUS A319-111, AIRBUS A318-111, AIRBUS A319-112, AIRBUS INDUSTRIE A319-111, AIRBUS A320-214]
Hawaiian Airlines Inc. [BOEING 717-200, BOEING 767-33A, BOEING 767-3CB, BOEING 767-332, BOEING 767-3G5]
Independence Air [BOMBARDIER INC CL-600-2B19]
JetBlue Airways [AIRBUS A320-232, EMBRAER ERJ 190-100 IGW, AIRBUS INDUSTRIE A320-232]
JetSuite Air [EMBRAER EMB-145LR, EMBRAER EMB-145XR, EMBRAER EMB-145EP, EMBRAER EMB-135LR, EMBRAER EMB-135ER]
Mesa Airlines Inc. [BOMBARDIER INC CL600-2D24, BOMBARDIER INC CL-600-2C10, BOMBARDIER INC CL-600-2B19, DEHAVILLAND DHC-8-202, CANADAIR CL-600-2B19]
Northwest Airlines Inc. [MCDONNELL DOUGLAS DC-9-51, BOEING 757-251, DOUGLAS DC-9-31, AIRBUS INDUSTRIE A320-212, AIRBUS INDUSTRIE A320-211]
PSA Airlines Inc. [BOMBARDIER INC CL-600-2B19, CANADAIR CL-600-2B19, BOMBARDIER INC CL-600-2C10, BOMBARDIER INC CL600-2D24, PIPER PA-28-180]
Pinnacle Airlines Inc. [BOMBARDIER INC CL-600-2B19, BOMBARDIER INC CL-600-2C10, BOMBARDIER INC CL600-2D24]
Skywest Airlines Inc. [BOMBARDIER INC CL-600-2B19, BOMBARDIER INC CL-600-2C10, EMBRAER EMB-120ER, BOMBARDIER INC CL600-2D24, CANADAIR CL-600-2B19]
Southwest Airlines Co. [BOEING 737-7H4, BOEING 737-3H4, BOEING 737-5H4, BOEING 737-3Q8, BOEING 737-3G7]
US Airways [AIRBUS INDUSTRIE A319-112, BOEING 737-4B7, AIRBUS INDUSTRIE A319-132, BOEING 737-3B7, BOEING 737-401]

In [0]:
stagemetrics.begin()

task_3_rdd(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_medium.csv", 
                                f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv", 
                                f"{dbfs_fileStore_prefix}/{prefix}_aircrafts.csv", "United States")
stagemetrics.end()

AirTran [BOEING 717-200, BOEING 737-7BD, BOEING 737-76N]
Alaska Airlines Inc. [BOEING 737-4Q8, BOEING 737-890, BOEING 737-490, BOEING 737-790, BOEING 737-990]
American Airlines Inc. [MCDONNELL DOUGLAS DC-9-82(MD-82), MCDONNELL DOUGLAS DC-9-83(MD-83), BOEING 757-223, AIRBUS INDUSTRIE A319-112, AIRBUS INDUSTRIE A319-132]
American Eagle Airlines Inc. [EMBRAER EMB-145LR, EMBRAER EMB-135KL, EMBRAER EMB-135LR, BOMBARDIER INC CL-600-2C10, SAAB-SCANIA SAAB 340B]
Atlantic Southeast Airlines [BOMBARDIER INC CL-600-2B19, EMBRAER EMB-145LR, EMBRAER EMB-145XR, BOMBARDIER INC CL-600-2C10, CANADAIR CL-600-2B19]
Comair [BOMBARDIER INC CL-600-2B19, CANADAIR CL-600-2B19, BOMBARDIER INC CL-600-2C10, BOMBARDIER INC CL600-2D24, PIPER PA-28-180]
Continental Air Lines Inc. [BOEING 737-524, BOEING 737-824, BOEING 737-3TO, BOEING 757-224, BOEING 737-724]
Delta Air Lines Inc. [MCDONNELL DOUGLAS AIRCRAFT CO MD-88, BOEING 757-232, BOEING 737-832, BOEING 717-200, BOEING 767-332]
Frontier Airlines Inc. [AIRBUS A319-111, AIRBUS A318-111, AIRBUS A319-112, AIRBUS INDUSTRIE A319-111, AIRBUS A320-214]
Hawaiian Airlines Inc. [BOEING 717-200, BOEING 767-33A, BOEING 767-3CB, BOEING 767-332, BOEING 767-3G5]
Independence Air [BOMBARDIER INC CL-600-2B19]
JetBlue Airways [AIRBUS A320-232, EMBRAER ERJ 190-100 IGW, AIRBUS INDUSTRIE A320-232]
JetSuite Air [EMBRAER EMB-145LR, EMBRAER EMB-145XR, EMBRAER EMB-145EP, EMBRAER EMB-135LR, EMBRAER EMB-135ER]
Mesa Airlines Inc. [BOMBARDIER INC CL600-2D24, BOMBARDIER INC CL-600-2C10, BOMBARDIER INC CL-600-2B19, DEHAVILLAND DHC-8-202, CANADAIR CL-600-2B19]
Northwest Airlines Inc. [MCDONNELL DOUGLAS DC-9-51, BOEING 757-251, DOUGLAS DC-9-31, AIRBUS INDUSTRIE A320-212, AIRBUS INDUSTRIE A320-211]
PSA Airlines Inc. [BOMBARDIER INC CL-600-2B19, CANADAIR CL-600-2B19, BOMBARDIER INC CL-600-2C10, BOMBARDIER INC CL600-2D24, PIPER PA-28-180]
Pinnacle Airlines Inc. [BOMBARDIER INC CL-600-2B19, BOMBARDIER INC CL-600-2C10, BOMBARDIER INC CL600-2D24]
Skywest Airlines Inc. [BOMBARDIER INC CL-600-2B19, BOMBARDIER INC CL-600-2C10, EMBRAER EMB-120ER, BOMBARDIER INC CL600-2D24, CANADAIR CL-600-2B19]
Southwest Airlines Co. [BOEING 737-7H4, BOEING 737-3H4, BOEING 737-5H4, BOEING 737-3Q8, BOEING 737-3G7]
US Airways [AIRBUS INDUSTRIE A319-112, BOEING 737-4B7, BOEING 737-3B7, BOEING 737-401, AIRBUS INDUSTRIE A319-132]

In [0]:
stagemetrics.begin()

task_3_rdd(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_large.csv", 
                                f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv", 
                                f"{dbfs_fileStore_prefix}/{prefix}_aircrafts.csv", "United States")
stagemetrics.end()

AirTran [BOEING 717-200, BOEING 737-7BD, BOEING 737-76N]
Alaska Airlines Inc. [BOEING 737-4Q8, BOEING 737-890, BOEING 737-490, BOEING 737-790, BOEING 737-990]
American Airlines Inc. [MCDONNELL DOUGLAS DC-9-82(MD-82), MCDONNELL DOUGLAS DC-9-83(MD-83), BOEING 757-223, AIRBUS INDUSTRIE A319-112, AIRBUS INDUSTRIE A319-132]
American Eagle Airlines Inc. [EMBRAER EMB-145LR, EMBRAER EMB-135KL, EMBRAER EMB-135LR, BOMBARDIER INC CL-600-2C10, SAAB-SCANIA SAAB 340B]
Atlantic Southeast Airlines [BOMBARDIER INC CL-600-2B19, EMBRAER EMB-145LR, EMBRAER EMB-145XR, BOMBARDIER INC CL-600-2C10, CANADAIR CL-600-2B19]
Comair [BOMBARDIER INC CL-600-2B19, CANADAIR CL-600-2B19, BOMBARDIER INC CL-600-2C10, BOMBARDIER INC CL600-2D24, PIPER PA-28-180]
Continental Air Lines Inc. [BOEING 737-524, BOEING 737-824, BOEING 737-3TO, BOEING 757-224, BOEING 737-724]
Delta Air Lines Inc. [MCDONNELL DOUGLAS AIRCRAFT CO MD-88, BOEING 757-232, BOEING 737-832, BOEING 717-200, BOEING 767-332]
Frontier Airlines Inc. [AIRBUS A319-111, AIRBUS A318-111, AIRBUS A319-112, AIRBUS INDUSTRIE A319-111, AIRBUS A320-214]
Hawaiian Airlines Inc. [BOEING 717-200, BOEING 767-33A, BOEING 767-3CB, BOEING 767-332, BOEING 767-3G5]
Independence Air [BOMBARDIER INC CL-600-2B19]
JetBlue Airways [AIRBUS A320-232, EMBRAER ERJ 190-100 IGW, AIRBUS INDUSTRIE A320-232]
JetSuite Air [EMBRAER EMB-145LR, EMBRAER EMB-145XR, EMBRAER EMB-145EP, EMBRAER EMB-135LR, EMBRAER EMB-135ER]
Mesa Airlines Inc. [BOMBARDIER INC CL600-2D24, BOMBARDIER INC CL-600-2C10, BOMBARDIER INC CL-600-2B19, DEHAVILLAND DHC-8-202, CANADAIR CL-600-2B19]
Northwest Airlines Inc. [MCDONNELL DOUGLAS DC-9-51, BOEING 757-251, DOUGLAS DC-9-31, AIRBUS INDUSTRIE A320-212, AIRBUS INDUSTRIE A320-211]
PSA Airlines Inc. [BOMBARDIER INC CL-600-2B19, CANADAIR CL-600-2B19, BOMBARDIER INC CL-600-2C10, BOMBARDIER INC CL600-2D24, PIPER PA-28-180]
Pinnacle Airlines Inc. [BOMBARDIER INC CL600-2D24, BOMBARDIER INC CL-600-2B19, BOMBARDIER INC CL-600-2C10]
Skywest Airlines Inc. [BOMBARDIER INC CL-600-2B19, BOMBARDIER INC CL-600-2C10, EMBRAER EMB-120ER, BOMBARDIER INC CL600-2D24, CANADAIR CL-600-2B19]
Southwest Airlines Co. [BOEING 737-7H4, BOEING 737-3H4, BOEING 737-5H4, BOEING 737-3Q8, BOEING 737-3G7]
US Airways [AIRBUS INDUSTRIE A319-112, BOEING 737-4B7, BOEING 737-3B7, AIRBUS INDUSTRIE A319-132, BOEING 737-401]

In [0]:
stagemetrics.begin()

task_3_rdd(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_massive.csv", 
                                f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv", 
                                f"{dbfs_fileStore_prefix}/{prefix}_aircrafts.csv", "United States")
stagemetrics.end()

AirTran [BOEING 717-200, BOEING 737-7BD, BOEING 737-76N]
Alaska Airlines Inc. [BOEING 737-4Q8, BOEING 737-890, BOEING 737-490, BOEING 737-790, BOEING 737-990]
American Airlines Inc. [MCDONNELL DOUGLAS DC-9-82(MD-82), MCDONNELL DOUGLAS DC-9-83(MD-83), BOEING 757-223, AIRBUS INDUSTRIE A319-112, AIRBUS INDUSTRIE A319-132]
American Eagle Airlines Inc. [EMBRAER EMB-145LR, EMBRAER EMB-135KL, EMBRAER EMB-135LR, BOMBARDIER INC CL-600-2C10, SAAB-SCANIA SAAB 340B]
Atlantic Southeast Airlines [BOMBARDIER INC CL-600-2B19, EMBRAER EMB-145LR, EMBRAER EMB-145XR, BOMBARDIER INC CL-600-2C10, CANADAIR CL-600-2B19]
Comair [BOMBARDIER INC CL-600-2B19, CANADAIR CL-600-2B19, BOMBARDIER INC CL-600-2C10, BOMBARDIER INC CL600-2D24, PIPER PA-28-180]
Continental Air Lines Inc. [BOEING 737-524, BOEING 737-824, BOEING 737-3TO, BOEING 737-724, BOEING 757-224]
Delta Air Lines Inc. [MCDONNELL DOUGLAS AIRCRAFT CO MD-88, BOEING 757-232, BOEING 737-832, BOEING 717-200, BOEING 767-332]
Frontier Airlines Inc. [AIRBUS A319-111, AIRBUS A318-111, AIRBUS A319-112, AIRBUS INDUSTRIE A319-111, AIRBUS A320-214]
Hawaiian Airlines Inc. [BOEING 717-200, BOEING 767-33A, BOEING 767-3CB, BOEING 767-332, BOEING 767-3G5]
Independence Air [BOMBARDIER INC CL-600-2B19]
JetBlue Airways [AIRBUS A320-232, EMBRAER ERJ 190-100 IGW, AIRBUS INDUSTRIE A320-232]
JetSuite Air [EMBRAER EMB-145LR, EMBRAER EMB-145XR, EMBRAER EMB-145EP, EMBRAER EMB-135LR, EMBRAER EMB-135ER]
Mesa Airlines Inc. [BOMBARDIER INC CL600-2D24, BOMBARDIER INC CL-600-2C10, BOMBARDIER INC CL-600-2B19, DEHAVILLAND DHC-8-202, CANADAIR CL-600-2B19]
Northwest Airlines Inc. [MCDONNELL DOUGLAS DC-9-51, BOEING 757-251, AIRBUS INDUSTRIE A320-212, DOUGLAS DC-9-31, AIRBUS INDUSTRIE A320-211]
PSA Airlines Inc. [BOMBARDIER INC CL-600-2B19, CANADAIR CL-600-2B19, BOMBARDIER INC CL-600-2C10, BOMBARDIER INC CL600-2D24, PIPER PA-28-180]
Pinnacle Airlines Inc. [BOMBARDIER INC CL600-2D24, BOMBARDIER INC CL-600-2B19, BOMBARDIER INC CL-600-2C10]
Skywest Airlines Inc. [BOMBARDIER INC CL-600-2B19, BOMBARDIER INC CL-600-2C10, EMBRAER EMB-120ER, BOMBARDIER INC CL600-2D24, CANADAIR CL-600-2B19]
Southwest Airlines Co. [BOEING 737-7H4, BOEING 737-3H4, BOEING 737-5H4, BOEING 737-3Q8, BOEING 737-3G7]
US Airways [AIRBUS INDUSTRIE A319-112, BOEING 737-4B7, BOEING 737-3B7, AIRBUS INDUSTRIE A319-132, BOEING 737-401]